In [32]:
import pandas as pd

# Load raw timetable data
hyd_pune = pd.read_csv("timetable_hyd_pune.csv")
pune_hyd = pd.read_csv("timetable_pune_hyd.csv")

# Assign proper column names
columns = ['SNo', 'Location', 'Primary', 'BoardingPoint', 'Time', 'Day', 'Latitude', 'Longitude', 'PointType']
hyd_pune.columns = columns
pune_hyd.columns = columns

# Convert 'Time' to proper time format
hyd_pune['Time'] = pd.to_datetime(hyd_pune['Time'], errors='coerce').dt.time
pune_hyd['Time'] = pd.to_datetime(pune_hyd['Time'], errors='coerce').dt.time

# Drop rows with missing time or GPS data
hyd_pune.dropna(subset=['Time', 'Latitude', 'Longitude'], inplace=True)
pune_hyd.dropna(subset=['Time', 'Latitude', 'Longitude'], inplace=True)

# Save back to original files (overwrite)
hyd_pune.to_csv("timetable_hyd_pune.csv", index=False)
pune_hyd.to_csv("timetable_pune_hyd.csv", index=False)


C:\Users\HP\AppData\Local\Temp\ipykernel_24892\4153668393.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hyd_pune['Time'] = pd.to_datetime(hyd_pune['Time'], errors='coerce').dt.time
C:\Users\HP\AppData\Local\Temp\ipykernel_24892\4153668393.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pune_hyd['Time'] = pd.to_datetime(pune_hyd['Time'], errors='coerce').dt.time


In [35]:
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic

# Load vehicle data and timetable
vehicle_df = pd.read_csv("vehicle.csv")
hyd_pune = pd.read_csv("timetable_hyd_pune.csv")

# Parse Timestamp
vehicle_df['Timestamp'] = pd.to_datetime(vehicle_df['Timestamp'])

# Ensure time column is datetime.time type
hyd_pune['Time'] = pd.to_datetime(hyd_pune['Time'], errors='coerce').dt.time

# Function to calculate nearest stop distance and time diff
def get_nearest_stop_info(lat, lon, timestamp):
    min_dist = float('inf')
    closest_time_diff = None
    for _, row in hyd_pune.iterrows():
        stop_loc = (row['Latitude'], row['Longitude'])
        current_loc = (lat, lon)
        dist = geodesic(current_loc, stop_loc).meters
        if dist < min_dist:
            min_dist = dist
            try:
                stop_time = datetime.combine(timestamp.date(), row['Time'])  # This is fixed
                closest_time_diff = (timestamp - stop_time).total_seconds() / 60  # in minutes
            except:
                closest_time_diff = None
    return min_dist, closest_time_diff

# Apply the function row-wise
vehicle_df[['DistanceToNearestStop_m', 'TimeDiffToStop_min']] = vehicle_df.apply(
    lambda row: pd.Series(get_nearest_stop_info(row['Latitude'], row['Longitude'], pd.to_datetime(row['Timestamp']))),
    axis=1
)

# Calculate stopped duration based on Speed = 0
vehicle_df['IsStopped'] = vehicle_df['Speed'] == 0
vehicle_df['StopGroup'] = (vehicle_df['IsStopped'] != vehicle_df['IsStopped'].shift()).cumsum()
vehicle_df['StoppedDuration'] = vehicle_df.groupby('StopGroup')['Timestamp'].transform(
    lambda x: (x.max() - x.min()).total_seconds() / 60
)
vehicle_df['StoppedDuration'] = vehicle_df['StoppedDuration'].where(vehicle_df['IsStopped'], 0)

# Save with added features
vehicle_df.to_csv("vehicle_with_features.csv", index=False)


C:\Users\HP\AppData\Local\Temp\ipykernel_24892\1022236198.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hyd_pune['Time'] = pd.to_datetime(hyd_pune['Time'], errors='coerce').dt.time
